In [8]:
import os
import pandas as pd
from datetime import timedelta
import re

In [ ]:
# Caminhos
BASE_DIR = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Brutos"
PATH_BOMBEIROS = os.path.join(BASE_DIR, "BOMBEIROS")
PATH_CIMEHGO = os.path.join(BASE_DIR, "CIMEHGO")
PATH_INMET = os.path.join(BASE_DIR, "INMET_GOIANIA")
PATH_RELATOS = os.path.join(BASE_DIR, "RELATOS")
PATH_EXPORT = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Processados"

# Bombeiros

In [10]:
def processar_bombeiros():
    colunas = ["N° DA OCORRÊNCIA", "Data", "ANO", "BAIRRO", "ESTADO", "ENDEREÇO", "FAIXA_ETARIA", "SEXO"]
    colunas_upper = [c.upper() for c in colunas]

    arquivos = [f for f in os.listdir(PATH_BOMBEIROS) if f.lower().endswith((".csv", ".xlsx"))]
    dfs = []

    for arquivo in arquivos:
        caminho = os.path.join(PATH_BOMBEIROS, arquivo)
        if arquivo.lower().endswith(".csv"):
            df = pd.read_csv(caminho, sep=";", encoding="latin1", engine="python")
        else:
            df = pd.read_excel(caminho)

        # normaliza os nomes das colunas
        df.columns = [c.strip().upper() for c in df.columns]

        # filtra apenas as colunas existentes
        colunas_existentes = [c for c in colunas_upper if c in df.columns]
        df_filtrado = df[colunas_existentes].copy()

        dfs.append(df_filtrado)

    df_final = pd.concat(dfs, ignore_index=True)

    # salva em Excel
    caminho_saida = os.path.join(PATH_EXPORT, "BOMBEIROS_TRATADO.xlsx")
    df_final.to_excel(caminho_saida, index=False)

# CIMEHGO

In [11]:
def processar_cimehgo():
    colunas = ["Estação", "Data/Hora(Local)", "Chuva(mm)", "ChuvaAcum.(mm)"]
    arquivos = [f for f in os.listdir(PATH_CIMEHGO) if f.lower().endswith((".csv", ".xlsx"))]

    dfs = []
    for arquivo in arquivos:
        caminho = os.path.join(PATH_CIMEHGO, arquivo)
        if arquivo.endswith(".csv"):
            df = pd.read_csv(caminho, sep=";", encoding="latin1", engine="python", skiprows=1)
        else:
            df = pd.read_excel(caminho)
            
            
        df = df[colunas]
        dfs.append(df)
        #df.columns = df.columns.str.strip()  # remove espaços extras
        #df = df[[c for c in colunas if c in df.columns]]  # seleciona apenas colunas existentes
        #dfs.append(df)

    if dfs:
        df_final = pd.concat(dfs, ignore_index=True)
        #df_final.to_csv(os.path.join(PATH_EXPORT, "CIMEHGO_TRATADO.csv"), index=False, sep=";", encoding="latin1")
        df_final.to_excel(os.path.join(PATH_EXPORT, "CIMEHGO_TRATADO.xlsx"))

# INMET

In [15]:
def processar_inmet():
    tabelas = []

    for arquivo in os.listdir(PATH_INMET):
        if arquivo.lower().endswith((".csv", ".xlsx", ".xls")):
            caminho = os.path.join(PATH_INMET, arquivo)
            print(f"Lendo: {arquivo}")

            if arquivo.lower().endswith(".csv"):
                df = pd.read_csv(caminho, sep=";", encoding="latin1", engine="python")
            else:
                df = pd.read_excel(caminho)

            tabelas.append(df)

    if not tabelas:
        print("Nenhum arquivo INMET encontrado!")
        return pd.DataFrame()

    # Concatenar arquivos diretamente
    df_inmet = pd.concat(tabelas, ignore_index=True)

    # Renomear coluna de precipitação se existir
    if "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)" in df_inmet.columns:
        df_inmet.rename(columns={"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "Chuva(mm)"}, inplace=True)

    # Extrair informações da coluna INFO se existir
    if "INFO" in df_inmet.columns:
        def extrair_info(pattern, texto):
            match = re.search(pattern, str(texto))
            return match.group(1).strip() if match else None

        df_inmet["ESTACAO"] = df_inmet["INFO"].apply(lambda x: extrair_info(r"ESTACAO:;([^/]+)", x))
        df_inmet["LATITUDE"] = df_inmet["INFO"].apply(lambda x: extrair_info(r"LATITUDE:;([-\d,.]+)", x))
        df_inmet["LONGITUDE"] = df_inmet["INFO"].apply(lambda x: extrair_info(r"LONGITUDE:;([-\d,.]+)", x))

        df_inmet["LATITUDE"] = df_inmet["LATITUDE"].str.replace(",", ".", regex=False).astype(float)
        df_inmet["LONGITUDE"] = df_inmet["LONGITUDE"].str.replace(",", ".", regex=False).astype(float)


    def converter_hora_utc(hora_utc):
        if pd.isna(hora_utc):
            return None
        # transformar em string e remover 'UTC' e espaços
        hora_str = str(hora_utc).replace("UTC", "").strip()
        # preencher com zeros à esquerda para garantir 4 dígitos
        hora_str = hora_str.zfill(4)
        hh = hora_str[:2]
        mm = hora_str[2:]
        return f"{hh}:{mm}"

    # aplicar ao DataFrame
    df_inmet["Hora"] = df_inmet["Hora UTC"].apply(converter_hora_utc)
    
    # Selecionar colunas finais de forma segura
    c_final = ["Data", "Hora", "DataHora", "Chuva(mm)", "LATITUDE", "LONGITUDE", "ESTACAO"]
    df_inmet = df_inmet.reindex(columns=c_final)

    # Salvar arquivo final
    #saida = os.path.join(PATH_EXPORT, "INMET_COMBINADO.csv")
    saida = os.path.join(PATH_EXPORT, "INMET_COMBINADO.xlsx")
    #df_inmet.to_csv(saida, sep=",", index=False, encoding="latin1")
    df_inmet.to_excel(saida)

    return df_inmet

# RELATOS

In [13]:
def processar_relatos():
    arquivos = [f for f in os.listdir(PATH_RELATOS) if f.lower().endswith((".csv", ".xlsx"))]

    if len(arquivos) <= 1:
        return

    dfs = []
    for arquivo in arquivos:
        caminho = os.path.join(PATH_RELATOS, arquivo)
        if arquivo.endswith(".csv"):
            df = pd.read_csv(caminho, sep=";", encoding="latin1", engine="python")
        else:
            df = pd.read_excel(caminho)
        dfs.append(df)

    df_final = pd.concat(dfs, ignore_index=True)
    df_final.to_csv(os.path.join(PATH_EXPORT, "RELATOS_TRATADO.csv"), index=False, sep=";", encoding="latin1")

# MAIN

In [16]:
if __name__ == "__main__":
    processar_bombeiros()
    processar_cimehgo()
    processar_inmet()
    processar_relatos()


c:\Users\Fujioka\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Lendo: INMET_GOIANIA_01-01-2020_A_31-12-2020.xlsx
Lendo: INMET_GOIANIA_01-01-2021_A_31-12-2021.xlsx
Lendo: INMET_GOIANIA_01-01-2022_A_31-12-2022.xlsx
Lendo: INMET_GOIANIA_01-01-2023_A_31-12-2023.xlsx
Lendo: INMET_GOIANIA_01-01-2024_A_31-12-2024.xlsx
Lendo: INMET_GOIANIA_01-01-2025_A_30-09-2025.xlsx
